In [1]:
import scipy.linalg as spla
import numpy as np
import itertools

from src.reductive_geometry import get_pauli_basis, VectorSpace, comm

$
\newcommand{\SU}{\mathrm{SU}}
\newcommand{\U}{\mathrm{U}}
\newcommand{\Stab}{\mathrm{Stab}}
\newcommand{\SO}{\mathrm{SO}}
\newcommand{\Sp}{\mathrm{Sp}}
\newcommand{\su}{\mathfrak{su}}
\newcommand{\so}{\mathfrak{so}}
\newcommand{\gl}{\mathfrak{gl}}
\renewcommand{\sp}{\mathfrak{sp}}
\renewcommand{\gg}{\mathfrak{g}}
\newcommand{\uu}{\mathfrak{u}}
\newcommand{\kk}{\mathfrak{k}}
\newcommand{\mm}{\mathfrak{m}}
\newcommand{\oo}{\mathfrak{o}}
\newcommand{\hh}{\mathfrak{h}}
\renewcommand{\tt}{\mathfrak{t}}
\newcommand{\Span}{\mathrm{span}}
$

Some code to easily format the final coefficients

In [2]:
def format_coeffs(coeffs, names):
    assert len(coeffs)==len(names)
    str_rep = ""
    for i, c_i in enumerate(coeffs):
        if not np.isclose(c_i,0.):
            str_rep = str_rep + f"{c_i:1.5f}*{names[i]} + "
    return str_rep[:-2]

A way to find the intersection subspace of several vector spaces

In [3]:
def intersection_basis(bases):
    """ Compute the basis for the intersection of the spans of sets of vectors. """
    # Convert sets to numpy arrays
    combined_complement=[]
    for b in bases:
        combined_complement.append(spla.null_space(b))
    # Concatenate the orthogonal complements
    combined_complement = np.hstack(combined_complement)
    intersection = spla.null_space(combined_complement.T)

    return intersection

A way to find the commutant given bases for Lie algebras $\kk\subset\gg$.

In [4]:
def find_commutant(g, k):
    total_kernel = []
    for k_i in k.basis:
        ad_g_i = []
        for g_j in g.basis:
            c = comm(k_i, g_j)
            ad_g_i.append(g.project_coeffs(c))
        ad_g_i = np.stack(ad_g_i)
        kernel = spla.null_space(1j*ad_g_i).T.real
        total_kernel.append(kernel)

    q,r = np.linalg.qr(total_kernel[0])
    bases = intersection_basis(total_kernel)
    return bases

A way to find a linearly independent set of vectors

In [43]:
def linearly_independent_set_svd(vectors):
    # Convert the list of vectors into a NumPy array
    A = np.array(vectors).T  # Transpose to get vectors as columns

    # Perform Singular Value Decomposition
    U, S, VT = np.linalg.svd(A)

    # Determine the rank of the matrix (number of non-zero singular values)
    rank = np.sum(S > 1e-10)  # Use a threshold to determine non-zero singular values

    # Select the first 'rank' columns of U (corresponding to non-zero singular values)
    independent_vectors = U[:, :rank]

    return independent_vectors.T  # Transpose back to original orientation

# Different homogeneous space $G/K$

In this notebook, we will explore how to find the tangent space parameterization $T_{G/K}\:gK \cong \mathfrak{m}$ for a given $G$ and $K$ and associated Lie algebras $\mathfrak{g}$ and $\mathfrak{k}$.

In [5]:
su2_dict = get_pauli_basis(1)
su4_dict = get_pauli_basis(2)
su2_names = list(su2_dict.keys())
su4_names = list(su4_dict.keys())
su2_vspace = VectorSpace(np.stack(list(su2_dict.values())))
su4_vspace = VectorSpace(np.stack(list(su4_dict.values())))

In [6]:
so4 = ['IY', 'XY', 'YI', 'YX', 'YZ', 'ZY']
so4_dict = dict(zip(so4, [su4_dict[p] for p in so4]))    
so4_names = list(so4_dict.keys())
so4_vspace = VectorSpace(np.stack(list(so4_dict.values())))

## 1.) $\SU(2)/\U(1)$

By inspection, we quickly find

$$
\begin{align}
\gg &= \Span\{X,Y,Z\} \\
\kk &= \Span\{Z\}\\
\mm &= \Span\{X,Y\}\\
\end{align}
$$
clearly the center is $\mathfrak{z}(\kk) = \Span\{Z\}$ and $\gg^\kk_o = 0$.

In [7]:
g = su2_vspace
k = VectorSpace(np.array([su2_dict["Z"]]))
m = VectorSpace(np.array([su2_dict["X"], su2_dict["Y"]]))

Determine the commutant

In [8]:
commutant = find_commutant(g, k)
for b in commutant.T:
    print(format_coeffs(b, su2_names))

1.00000*Z 


Confirms that the commutant equals

$$
\gg^\kk = \{Z\}
$$

## 2.) $\SU(4)/\U(2)$ (spin 1/2)

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg &= i\,\Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} 
\end{align}
$$

It's clear that

$$
\begin{align}
    \kk &= i\,\Span\{XI+IX, YI+IY, ZI + IZ\}
\end{align}
$$

satisfies the commutation relations of $\su(2)$. We now have a trivial center $\mathfrak{z}(\kk)=0$. To find the space $\mm$, we use a nullspace method to find the elements of $\gg$ that are orthogonal to $\kk$. 

In [9]:
I = np.eye(2, dtype=complex)
g = VectorSpace(su4_vspace.basis)
# Create the su(2) basis within su(4)
k = VectorSpace(np.stack([np.kron(su2_dict['X'], I) + np.kron(I, su2_dict['X']),
                      np.kron(su2_dict['Y'], I) + np.kron(I, su2_dict['Y']),
                      np.kron(su2_dict['Z'], I) + np.kron(I, su2_dict['Z'])]) / 2)
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
    
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)

# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, su4_names))
    

Dimension of m: 12
-0.70711*IX + 0.70711*XI 
1.00000*XX 
1.00000*XY 
1.00000*XZ 
-0.70711*IY + 0.70711*YI 
1.00000*YX 
1.00000*YY 
1.00000*YZ 
-0.70711*IZ + 0.70711*ZI 
1.00000*ZX 
1.00000*ZY 
1.00000*ZZ 


Hence

$$
\begin{align}
 \mm=i\,\Span\{XX,XY,XZ,YX,YY,YZ, ZX,ZY,ZZ, XI-IX, YI-IY, ZI - IZ\}
\end{align}
$$

We can find the commutant by determining the kernel of $\mathrm{ad}_Y$ for all $Y \in \kk$ within $\gg$.

In [10]:
commutant = find_commutant(g, k)
for b in commutant.T:
    print(format_coeffs(b, su4_names))
pbasis = get_pauli_basis(2)
print(pbasis['XX']+pbasis['YY']+pbasis['ZZ'] + 0.5 * np.eye(4))

0.57735*XX + 0.57735*YY + 0.57735*ZZ 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


Hence we find that the commutant is given by

$$
\gg^\kk = \{\mathrm{SWAP}\}
$$

which is also a direct consequence of the Schur-Weyl duality. Clearly $\mathfrak{z}(\kk)=0$, hence $\gg^\kk_o=\{\mathrm{SWAP}\}$

## 3a.) $\SU(4) / (I_2 \times \SU(2))$

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
\gg &= i\,\Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\,\Span\{0_2\oplus X, 0_2\oplus Y,0_2\oplus Z\}
\end{align}
$$
where $0_2$ is a zero matrix of size $2\times 2$. The center is trivial $\mathfrak{z}(\kk)=0$

In [11]:
I = np.eye(2, dtype=complex)
g = VectorSpace(su4_vspace.basis)
k = []
for term in su2_vspace.basis:
    k.append(np.block([[np.zeros((2,2)), np.zeros((2,2))],
                       [np.zeros((2,2)), term]]))
k = VectorSpace(np.stack(k))
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)
# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, su4_names))

Dimension of m: 12
1.00000*XI 
1.00000*XX 
1.00000*XY 
1.00000*XZ 
1.00000*YI 
1.00000*YX 
1.00000*YY 
1.00000*YZ 
1.00000*ZI 
0.70711*IX + 0.70711*ZX 
0.70711*IY + 0.70711*ZY 
0.70711*IZ + 0.70711*ZZ 


Hence

$$
\begin{align}
     \mm=i\,\Span\{XI, XX ,XY ,XZ ,YI ,YX ,YY ,YZ ,ZI ,IX + ZX ,IY + ZY ,IZ + ZZ \}
\end{align}
$$

In [12]:
commutant = find_commutant(g, k)
for b in commutant.T:
    print(format_coeffs(b, su4_names))
pbasis = get_pauli_basis(2)

0.00117*IX + -0.54495*IY + -0.45058*IZ + 0.00117*ZX + -0.54495*ZY + -0.45058*ZZ 
-0.57861*IX + -0.25975*IY + 0.31264*IZ + -0.57861*ZX + -0.25975*ZY + 0.31264*ZZ 
0.40646*IX + -0.36818*IY + 0.44635*IZ + 0.40646*ZX + -0.36818*ZY + 0.44635*ZZ 
-1.00000*ZI 


Hence the commutant is given by

$$
\begin{align}
\gg^\kk = \{&a_1(IX+ZX) + b_1(IY+ZY) + c_1(IZ+ZZ),\\
&a_2(IX+ZX) + b_2(IY+ZY) + c_2(IZ+ZZ),\\
&a_3(IX+ZX) + b_3(IY+ZY) + c_3(IZ+ZZ),\\
&-ZI\}
\end{align}
$$

## 3b.) $\SO(4) / (I_2 \times \SO(2))$

The basis for $\so(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg = i\:\Span\{IY, XY, YI, YX, YZ, ZY\}
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\,\Span\{0_2\oplus Y\}
\end{align}
$$
where $0_2$ is a zero matrix of size $2\times 2$. The center is trivial $\mathfrak{z}(\kk)=0$

In [13]:
I = np.eye(2, dtype=complex)
g = VectorSpace(so4_vspace.basis)
k = []
k.append(np.block([[np.zeros((2,2)), np.zeros((2,2))],
                   [np.zeros((2,2)), su2_dict['Y']]]))
k = VectorSpace(np.array(k))
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)
# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, so4_names))

Dimension of m: 5
1.00000*XY 
1.00000*YI 
1.00000*YX 
1.00000*YZ 
0.70711*IY + 0.70711*ZY 


Hence

$$
\begin{align}
     \mm=i\,\Span\{XY,YI ,YX ,YZ, IY + ZY\}
\end{align}
$$

Determine the commutant via a null space method

In [14]:
commutant = find_commutant(g, k)
for b in commutant.T:
    print(format_coeffs(b, so4_names))
pbasis = get_pauli_basis(2)
assert np.allclose(comm(k[0],pbasis['IY']), np.zeros([4]*2))
assert np.allclose(comm(k[0],pbasis['ZY']), np.zeros([4]*2))

1.00000*IY 
1.00000*ZY 


Hence the commutant is 

$$
\gg^\kk = \{IY, ZY\}
$$

## 3c.) $\SO(4) / (1 \times \SO(2) \times 1)$

The basis for $\so(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg = i\:\Span\{IY, XY, YI, YX, YZ, ZY\}
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\,\Span\{0\oplus Y\oplus0\}
\end{align}
$$

The center is trivial $\mathfrak{z}(\kk)=0$

In [15]:
I = np.eye(2, dtype=complex)
g = VectorSpace(so4_vspace.basis)
k = []
k.append(np.block([[0,0,0,0],[0,0,-1j,0],[0,1j,0,0],[0,0,0,0]]))
k = VectorSpace(np.array(k))
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)
# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, so4_names))


Dimension of m: 5
0.70711*IY + 0.50000*XY + 0.50000*YX 
1.00000*YI 
-0.70711*IY + 0.50000*XY + 0.50000*YX 
1.00000*YZ 
1.00000*ZY 


Hence

$$
\begin{align}
     \mm=i\,\Span\{\sqrt{2}IY+ XY+YX,-\sqrt{2}IY+ XY+YX, YI,YZ, ZY \}
\end{align}
$$

In [16]:
commutant = find_commutant(g, k)
for b in commutant.T:
    print(format_coeffs(b, so4_names))
pbasis = get_pauli_basis(2)
assert np.allclose(comm(k[0],pbasis['XY']), np.zeros([4]*2))
assert np.allclose(comm(k[0],pbasis['YX']), np.zeros([4]*2))

-1.00000*XY 
-1.00000*YX 


Hence the commutant is

$$
\gg^\kk = \{XY, YX\}
$$

## 4.) $\SU(4)/\SU(2)$ (spin 3/2)

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
\gg &= i\,\Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

A basis for $\kk$ is 

$$
\begin{align}
    \kk &= i\:\Span\{S_x, S_y, S_z\}
\end{align}
$$

with 

$$
\begin{align}
S_x &= \begin{bmatrix}
    0 & \sqrt{3} & 0 & 0 \\
    \sqrt{3} & 0 & 2 & 0 \\
    0 & 2 & 0 & \sqrt{3} \\
    0 & 0 & \sqrt{3} & 0 \\
    \end{bmatrix},\quad S_y = \begin{bmatrix}
    0 & -i\sqrt{3} & 0 & 0 \\
    i\sqrt{3} & 0 & -2i & 0 \\
    0 & 2i & 0 & -i\sqrt{3} \\
    0 & 0 & i\sqrt{3} & 0 \\
    \end{bmatrix},\quad S_z = \begin{bmatrix}
    3 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & -3 \\
    \end{bmatrix},
\end{align}
$$

we find $\mm$ via the null space method.

In [17]:
g = VectorSpace(su4_vspace.basis)
Sx = np.array([[0, np.sqrt(3), 0, 0],
               [np.sqrt(3), 0, 2, 0],
               [0, 2, 0, np.sqrt(3)],
               [0, 0, np.sqrt(3), 0]]) / 2
Sy = np.array([[0, -1j * np.sqrt(3), 0, 0],
               [1j * np.sqrt(3), 0, -2j, 0],
               [0, 2j, 0, -1j * np.sqrt(3)],
               [0, 0, 1j * np.sqrt(3), 0]]) / 2
Sz = np.array([[3, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, -3]]) / 2
k = VectorSpace(np.stack([Sx, Sy, Sz]))

subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
    
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)

# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, su4_names))
    

Dimension of m: 12
1.00000*XI 
-0.44721*IX + 0.88730*XX + -0.11270*YY 
0.44721*IY + 0.88730*XY + 0.11270*YX 
1.00000*XZ 
1.00000*YI 
-0.44721*IY + 0.11270*XY + 0.88730*YX 
-0.44721*IX + -0.11270*XX + 0.88730*YY 
1.00000*YZ 
-0.89443*IZ + 0.44721*ZI 
1.00000*ZX 
1.00000*ZY 
1.00000*ZZ 


Hence,

$$
\begin{align}
 \mm = &i\,\Span\{XI 
-aIX + bXX + -cYY ,
aIY + bXY + cYX ,
XZ,YI ,
-aIY + cXY + bYX ,\\
&-aIX + -cXX + bYY ,
YZ ,
-dIZ + aZI, 
ZX ,ZY ,ZZ \}
\end{align}
$$

With the coefficients $a= \frac{1}{\sqrt{5}}$, $b=\frac{5+\sqrt{15}}{10}$, $c = \frac{5-\sqrt{15}}{10}$ and $d=\frac{2}{\sqrt{5}}$ (by plugging in the above floating point numbers into Wolfram Alpha). We find $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

In [18]:
commutant = find_commutant(g, k)
print(commutant)

[]


Hence the commutant is empty

## 5.) $\SU(4)/\Sp(2)$

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
\gg &= \Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

Since $\SU(4)/\Sp(2)$ is a symmetric space, we can find a basis for $\Sp(2)$ by defining an involution $\theta:\gg\to\gg$ such that $\kk$ and $\mm$ correspond to the positive and negative eigenspaces of $\theta$, respectively. The involution that results in the split $\su(4) = \mm\oplus\sp(2)$ is 

$$
\begin{align}
    \theta(A) = J \cdot A^* \cdot J^T
\end{align}
$$
where 
$$
\begin{align}
J = \begin{pmatrix}
0 & I_2 \\
-I_2 & 0
\end{pmatrix}
\equiv iYI
\end{align}
$$

In [51]:
# Create a random matrix in SU(4)
standard_basis = np.eye(15)
elements_in_k = []
for i in range(15):
    coeffs = standard_basis[i]
    mat = 1j * su4_vspace.construct_op(coeffs)
    g = VectorSpace(su4_vspace.basis)

    
    # Use the involution to split off k
    J = np.kron(1j * su2_dict['Y'], np.eye(2)) * np.sqrt(2)
    theta_mat = J @ mat.conj() @ J.T

    # m is the negative eigenspace
    m = 0.5 * (mat - theta_mat)
    # k is the positive eigenspace
    k = 0.5 * (mat + theta_mat)

    coeffs_m = g.project_coeffs(m).imag
    coeffs_k = g.project_coeffs(k).imag
    elements_in_k.append(coeffs_k)
elements_in_k = np.stack(elements_in_k)
basis = linearly_independent_set_svd(elements_in_k)
for b in basis:
    print(format_coeffs(b, su4_names))

1.00000*ZX 
1.00000*XZ 
1.00000*YI 
1.00000*YX 
1.00000*ZZ 
1.00000*YZ 
1.00000*IY 
1.00000*ZI 
1.00000*XI 
1.00000*XX 


Hence we find the bases

$$
\begin{align}
\kk &= i\,\Span\{IY ,XI ,XX ,XZ ,YI,YX ,YZ ,ZI ,ZX ,ZZ \}\\
\mm &= i\,\Span\{IX,IZ,XY,YY,ZY \}
\end{align}
$$

with $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

In [20]:
k = VectorSpace(np.stack([su4_dict['IY'],su4_dict['XI'],su4_dict['XX'],
                          su4_dict['XZ'],su4_dict['YI'],su4_dict['YX'],
                          su4_dict['YZ'],su4_dict['ZI'],su4_dict['ZX'], su4_dict['ZZ']]))
commutant = find_commutant(g, k)
print(commutant)

[]


Hence the commutant is empty

## 6.) $\SU(4) / (\SU(2)\times\SU(2))$

The basis for $\su(4)$ is given by tensor products of Pauli matrices.

$$
\begin{align}
    \gg &= \Span\{IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ\} \\
\end{align}
$$

We see that $\kk$ is given by
$$
\begin{align}
    \kk &= i\:\Span\{XI, YI, ZI, IX, IY, IZ\}\\
\end{align}
$$
hence we can immediately conclude that
$$
\begin{align}
    \mm = i\:\Span\{XX, YY, ZZ, XY, XZ, YX, YZ, ZX, ZY\}\\
\end{align}
$$
which is the orthogonal complement of $\kk$ in $\gg$. Both centers are trivial, $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

In [21]:
k = VectorSpace(np.stack([su4_dict['IY'],su4_dict['YI'],su4_dict['IX'],
                          su4_dict['XI'],su4_dict['IZ'],su4_dict['ZI']]))
commutant = find_commutant(g, k)
print(commutant)

[]


The commutant is empty as expected.

## 7.) $\SO(4)/\SU(2)$

A basis for $\gg$ is given by the set of anti-symmetric matrices,

$$
\begin{align}
    \gg = i\:\Span\{IY, XY, YI, YX, YZ, ZY\}
\end{align}
$$

It is quickly confirmed that 

$$
\begin{align}
    \kk &= i\:\Span\{K_1, K_2, K_3\}\\
\end{align}
$$

with

$$
\begin{align}
    K_1 &=(YI+IY),\quad K_2=XY + YX,\quad K_3 = YZ + ZY
\end{align}
$$

satisfies the commutation relations of $\su(2)$. Since $\SO(4)/\SU(2)$ is a symmetric space, we could either use an involution to find $\mm$ or construct the orthogonal complement of $\kk$. Here, we do the latter.

In [22]:
I = np.eye(2, dtype=complex)
g = VectorSpace(so4_vspace.basis)
# Create the su(2) basis within su(4)
k = np.stack([np.kron(su2_dict['Y'], I) + np.kron(I, su2_dict['Y']),
              np.kron(su2_dict['Y'], su2_dict['X']) + np.kron(su2_dict['X'], su2_dict['Y']),
              np.kron(su2_dict['Z'], su2_dict['Y']) + np.kron(su2_dict['Y'], su2_dict['Z'])]) / 2
k = VectorSpace(k)
subspace_k = []
# Embed k in g
for i, p in enumerate(k):
    subspace_k.append(g.project_coeffs(p))
    
# Find the kernel of k
k_ortho = spla.null_space(np.stack(subspace_k))
m = np.einsum("in,ijk->njk", k_ortho, g.basis)

# Print the basis of m in terms of the pauli matrices
print("Dimension of m:", m.shape[0])
for m_i in m:
    coeffs = g.project_coeffs(m_i).real
    print(format_coeffs(coeffs, so4_names))
    

Dimension of m: 3
-0.70711*XY + 0.70711*YX 
0.50000*IY + -0.50000*YI + 0.50000*YZ + -0.50000*ZY 
0.50000*IY + -0.50000*YI + -0.50000*YZ + 0.50000*ZY 


Hence we obtain

$$
\begin{align}
    \mm = i\:\Span\{YX-XY, IY-YI + YZ-ZY, IY-YI + ZY- YZ\}
\end{align}
$$

Both centers are trivial, $\gg^\kk_o=0$ and $\mathfrak{z}(\kk)=0$. 

In [23]:
commutant = find_commutant(g, k)
print(commutant)

[]


Hence the commutant is empty

## 8.) $\SU(4)/\SU(3)$

In [50]:
# Create a random matrix in SU(4)
standard_basis = np.eye(15)
elements_in_k = []
for i in range(15):
    coeffs = standard_basis[i]
    mat = 1j * su4_vspace.construct_op(coeffs)
    g = VectorSpace(su4_vspace.basis)

    # Use the involution to split off k
    p,q = (1,3)
    Ip = np.eye(p)
    Iq = np.eye(q)
    zero_pq = np.zeros((p, q))
    Ipq = np.block([[-Ip, zero_pq],
                    [zero_pq.T, Iq]])

    theta_mat = Ipq @ mat @ Ipq

    # m is the negative eigenspace
    m = 0.5 * (mat - theta_mat)
    # k is the positive eigenspace
    k = 0.5 * (mat + theta_mat)

    coeffs_m = g.project_coeffs(m).imag
    coeffs_k = g.project_coeffs(k).imag
    elements_in_k.append(coeffs_k)
elements_in_k = np.stack(elements_in_k)
basis = linearly_independent_set_svd(elements_in_k)
for b in basis:
    print(format_coeffs(b, su4_names))

-0.70711*YI + 0.70711*YZ 
0.70711*IY + -0.70711*ZY 
-0.70711*XY + 0.70711*YX 
1.00000*ZZ 
1.00000*IZ 
-0.70711*XI + 0.70711*XZ 
1.00000*ZI 
-0.70711*IX + 0.70711*ZX 
-0.70711*XX + -0.70711*YY 


From which we quickly find

$$
\begin{align}
    \mm &= i\:\Span\{(I+Z)X, X(I+Z),(I+Z)Y, Y(I+Z),XY+YX, XX-YY\}\\
    \kk &= i\:\Span\{(I-Z)X, (I-Z)Y, X(I-Z), Y(I-Z), XY-YX, XX+YY, IZ, ZI, ZZ\}
\end{align}
$$


In [25]:
k = VectorSpace(np.stack([su4_dict['IX']+ su4_dict['ZX'],su4_dict['IY']+ su4_dict['ZY'],
                          su4_dict['XI']+ su4_dict['XZ'],su4_dict['YI']+ su4_dict['YZ'],
                          su4_dict['XY']- su4_dict['YX'],su4_dict['XX']+su4_dict['YY'],
                          su4_dict['IZ'], su4_dict['ZI'],su4_dict['ZZ']]))
commutant = find_commutant(g, k)
for b in commutant.T:
    print(format_coeffs(b, su4_names))

0.57735*IZ + 0.57735*ZI + -0.57735*ZZ 


Hence the commutant is 

$$
\gg^\kk = \{IZ+ZI-ZZ\}
$$

## 9.) $\SO(4) / \SO(3)$ 

In [69]:
# Create a random matrix in SU(4)
standard_basis = np.eye(6)
elements_in_k = []
for i in range(6):
    coeffs = standard_basis[i]
    mat = 1j * so4_vspace.construct_op(coeffs)
    g = VectorSpace(so4_vspace.basis)

    # Use the involution to split off k
    p,q = (1,3)
    Ip = np.eye(p)
    Iq = np.eye(q)
    zero_pq = np.zeros((p, q))
    Ipq = np.block([[-Ip, zero_pq],
                    [zero_pq.T, Iq]])

    theta_mat = Ipq @ mat @ Ipq

    # m is the negative eigenspace
    m = 0.5 * (mat - theta_mat)
    # k is the positive eigenspace
    k = 0.5 * (mat + theta_mat)

    coeffs_m = g.project_coeffs(m).imag
    coeffs_k = g.project_coeffs(k).imag
    elements_in_k.append(coeffs_k)
elements_in_k = np.stack(elements_in_k)
basis = linearly_independent_set_svd(elements_in_k)
for b in basis:
    print(format_coeffs(b, so4_names))

-0.70711*XY + 0.70711*YX 
-0.70711*YI + 0.70711*YZ 
-0.70711*IY + 0.70711*ZY 


From which we quickly find

$$
\begin{align}
    \mm &= i\:\Span\{(I+Z)Y, Y(I+Z), XY+YX\}\\
    \kk &= i\:\Span\{(I-Z)Y, Y(I-Z), XY-YX\}
\end{align}
$$


In [71]:
b = get_pauli_basis(2)
k = VectorSpace(np.stack([b['IY'] - b['ZY'], b['YI'] - b['YZ'], b['XY']-b['YX']]))
commutant = find_commutant(g, k)
print(commutant)
for b in commutant.T:
    print(format_coeffs(b, su4_names))

[]


# $\SU(8) / \mathrm{S}(\U(2)\times\U(6))$ 

In [72]:
su8_dict = get_pauli_basis(3)
su8_names = list(su8_dict.keys())
su8_vspace = VectorSpace(np.stack(list(su8_dict.values())))

In [73]:
# Create a random matrix in SU(4)
standard_basis = np.eye(63)
elements_in_k = []
elements_in_m = []
for i in range(63):
    coeffs = standard_basis[i]
    mat = 1j * su8_vspace.construct_op(coeffs)
    g = VectorSpace(su8_vspace.basis)

    # Use the involution to split off k
    p,q = (2,6)
    Ip = np.eye(p)
    Iq = np.eye(q)
    zero_pq = np.zeros((p, q))
    Ipq = np.block([[-Ip, zero_pq],
                    [zero_pq.T, Iq]])

    theta_mat = Ipq @ mat @ Ipq

    # m is the negative eigenspace
    m = 0.5 * (mat - theta_mat)
    # k is the positive eigenspace
    k = 0.5 * (mat + theta_mat)

    coeffs_m = g.project_coeffs(m).imag
    coeffs_k = g.project_coeffs(k).imag
    elements_in_k.append(coeffs_k)
    elements_in_m.append(coeffs_m)
    
elements_in_k = np.stack(elements_in_k)
elements_in_m = np.stack(elements_in_m)
print("k")
print("_"*10)
basis = linearly_independent_set_svd(elements_in_k)
k_basis = []
for b in basis:
    k_basis.append(g.construct_op(b))
    print(format_coeffs(b, su8_names))
k = VectorSpace(np.stack(k_basis))

print("m")
print("_"*10)
basis = linearly_independent_set_svd(elements_in_m)
for b in basis:
    print(format_coeffs(b, su8_names))

k
__________
-1.00000*IIX 
1.00000*ZIX 
-1.00000*IIZ 
1.00000*ZIZ 
-0.70711*IXX + 0.70711*ZXX 
-0.70711*IXZ + 0.70711*ZXZ 
-0.70711*IYX + 0.70711*ZYX 
-0.70711*IYZ + 0.70711*ZYZ 
-1.00000*IZX 
1.00000*ZZX 
-1.00000*IZZ 
1.00000*ZZZ 
-0.70711*XIX + 0.70711*XZX 
-0.70711*XIZ + 0.70711*XZZ 
-0.70711*XXX + -0.70711*YYX 
-0.70711*XXZ + -0.70711*YYZ 
-0.70711*XYX + 0.70711*YXX 
-0.70711*XYZ + 0.70711*YXZ 
-0.70711*XIY + 0.70711*XZY 
-0.70711*XXI + -0.70711*YYI 
-0.70711*YIX + 0.70711*YZX 
-0.70711*YIZ + 0.70711*YZZ 
-0.70711*XYY + 0.70711*YXY 
0.70711*XII + -0.70711*XZI 
0.70711*XXY + 0.70711*YYY 
0.70711*XYI + -0.70711*YXI 
-0.70711*YIY + 0.70711*YZY 
-0.70711*YII + 0.70711*YZI 
-1.00000*IIY 
1.00000*ZIY 
-0.70711*IXI + 0.70711*ZXI 
-0.70711*IXY + 0.70711*ZXY 
-0.70711*IYI + 0.70711*ZYI 
-0.70711*IYY + 0.70711*ZYY 
-1.00000*IZI 
1.00000*ZZI 
-1.00000*IZY 
1.00000*ZZY 
-1.00000*ZII 
m
__________
-0.70711*IXI + -0.70711*ZXI 
-0.70711*IXY + -0.70711*ZXY 
-0.70711*IYI + -0.70711*ZYI 
-0.70711*I

In [74]:
print(g.shape)
print(k.shape)
commutant = find_commutant(g, k)
print(commutant)

(63, 8, 8)
(39, 8, 8)
[]
